In [1]:
from preprocess_synthea import *
import pandas as pd
import os

#can read/write only csvs or use gspread to write data as gsheets
#requires setup in google cloud platform and credentials.json and token.json files in current directory
from gsheet_requests import *

In [2]:
#expected synthea csv files:

#patients.csv
#allergies.csv
#careplans.csv
#conditions.csv
#devices.csv
#encounters.csv
#imaging_studies.csv
#immunizations.csv
#medications.csv
#observations.csv
#procedures.csv

In [3]:
#directory for synthea output/csv
in_path = "PATH_TO/output/csv/"

#write out path
out_path = "PATH_TO/my_data/"

In [4]:
patient_df, allergy_df, careplan_df, conditions_df, device_df, encounter_df, imaging_study_df, immunization_df, medication_df, observation_df, procedure_df = load_synthea(in_path)

In [5]:
#specify case criteria with boolean masks
#example filters:

#filter1: presence of Pulmonary emphysema or Chronic obstructive bronchitis (COPD)
conditions = set(["Pulmonary emphysema (disorder)", "Chronic obstructive bronchitis (disorder)"])
patient_copd = set(conditions_df[conditions_df["DESCRIPTION"].isin(conditions)]["PATIENT"].unique())

#filter2: female
patient_female = set(patient_df[patient_df["GENDER"] == "F"]["PATIENT"])

#filter3: record of covid-19 immunization
patient_covid_jab = set(immunization_df[immunization_df["DESCRIPTION"].str.contains("SARS-COV-2")]["PATIENT"].unique())

In [6]:
case_patients = generate_cohort(patient_df, 
                                #filter patient sets as arguments:
                                patient_copd, patient_female, patient_covid_jab)

Total patients meeting criteria: 88
Total patients not meeting criteria: 11558


In [7]:
cohort = generate_cohort_sample(80, 80, case_patients, patient_df)

In [8]:
patient_df, allergy_df, careplan_df, conditions_df, device_df, imaging_study_df, immunization_df, medication_df, observation_df, procedure_df = filter_cohort_data(cohort, 
                                                                                                                                                                   case_patients,
                                                                                                                                                                   patient_df,
                                                                                                                                                                   allergy_df,
                                                                                                                                                                   careplan_df,
                                                                                                                                                                   conditions_df,
                                                                                                                                                                   device_df,
                                                                                                                                                                   encounter_df,
                                                                                                                                                                   imaging_study_df,
                                                                                                                                                                   immunization_df,
                                                                                                                                                                   medication_df,
                                                                                                                                                                   observation_df,
                                                                                                                                                                   procedure_df)

In [9]:
ehr_df = join_synthea_tables(allergy_df,
                               careplan_df,
                               conditions_df,
                               device_df,
                               encounter_df,
                               imaging_study_df,
                               immunization_df,
                               medication_df,
                               observation_df,
                               procedure_df)

In [12]:
patient_df.head()

,PATIENT,BIRTHDATE,DEAD,AGE,FIRST,LAST,MARITAL,RACE,GENDER,CASE
0,166a2744-bd35-a60e-2b96-a39ebbf516fe,1990-12-15,0,33,Ji399,Bednar518,S,white,F,1
1,f2fcbb85-75dd-a75e-7f52-4543ea004578,1967-01-07,0,57,Temeka297,Jacobs452,D,asian,F,1
2,7d230b0b-67d7-d9e3-bf82-9fc9a3a18af4,1974-07-15,0,49,Lorraine478,Anderson154,M,white,F,0
3,b0f1e7cc-b2b2-d518-7332-438b4cedeeab,1937-02-17,0,86,Porfirio146,Casper496,M,white,M,0
4,fea33fbf-d04d-b8f1-5c01-efe79a59fae7,1967-07-08,0,56,Yoko498,Halvorson124,S,white,F,1


In [13]:
ehr_df

,CLASS,PATIENT,DATE,SYSTEM,DESCRIPTION,CODE,VALUE,UNITS,TYPE
0,allergy,f2fcbb85-75dd-a75e-7f52-4543ea004578,1967-10-25,SNOMED-CT,Allergy to substance (finding),419199007,NaN,NaN,NaN
1,allergy,f2fcbb85-75dd-a75e-7f52-4543ea004578,1967-10-25,SNOMED-CT,Tree nut (substance),442571000124108,NaN,NaN,NaN
2,allergy,7d230b0b-67d7-d9e3-bf82-9fc9a3a18af4,1977-03-05,SNOMED-CT,Allergy to substance (finding),419199007,NaN,NaN,NaN
3,allergy,7d230b0b-67d7-d9e3-bf82-9fc9a3a18af4,1977-03-05,RxNorm,Lisinopril,29046,NaN,NaN,NaN
4,allergy,7d230b0b-67d7-d9e3-bf82-9fc9a3a18af4,1977-03-05,RxNorm,Ibuprofen,5640,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
113659,observation,19c74dd1-fb55-17de-7096-070be4889c6b,1964-04-05,LOINC,Hemoglobin [Mass/volume] in Blood,718-7,10.9,g/dL,numeric
113662,observation,19c74dd1-fb55-17de-7096-070be4889c6b,1964-04-05,LOINC,Erythrocytes [#/volume] in Blood by Automated ...,789-8,5.3,10*6/uL,numeric
113549,observation,19c74dd1-fb55-17de-7096-070be4889c6b,1963-03-24,LOINC,Hematocrit [Volume Fraction] of Blood by Autom...,4544-3,36.3,%,numeric
113504,observation,19c74dd1-fb55-17de-7096-070be4889c6b,1961-03-12,LOINC,Total score [DAST-10],82667-7,0.0,{score},numeric


In [ ]:
#write joined cohort files
patient_df.to_csv(out_path + "patients.csv", index=False)
ehr_df.to_csv(out_path + "ehr_records.csv", index=False)

In [31]:
#write all cohort files
patient_df.to_csv(out_path + "patients.csv", index=False)
allergy_df.to_csv(out_path + "allergies.csv", index=False)
careplan_df.to_csv(out_path + "careplans.csv", index=False)
conditions_df.to_csv(out_path + "conditions.csv", index=False)
device_df.to_csv(out_path + "devices.csv", index=False)
encounter_df.to_csv(out_path + "encounters.csv", index=False)
imaging_study_df.to_csv(out_path + "imaging_studies.csv", index=False)
immunization_df.to_csv(out_path + "immunizations.csv", index=False)
medication_df.to_csv(out_path + "medications.csv", index=False)
observation_df.to_csv(out_path + "observations.csv", index=False)
procedure_df.to_csv(out_path + "procedures.csv", index=False)

In [10]:
#write files as gsheet tabs
#note gsheet allows a max number of cells around 5million

#gspread client
gc = gspread_client()
#gsheet document id
doc_id = ""

#read tab worksheets
patient_ws = read_sheet(doc_id, "patients", gc)[1]
ehr_ws = read_sheet(doc_id, "ehr", gc)[1]

#write dataframes to tabs
write_sheet(patient_df, patient_ws)
write_sheet(ehr_df, ehr_ws)

serving sheet: Untitled spreadsheet, patients
serving sheet: Untitled spreadsheet, ehr
